<a href="https://colab.research.google.com/github/limjunxiann/Furniture-Purchasing-System/blob/main/SecondHandCarDashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio seaborn matplotlib pandas

In [ ]:
import os
import time
import pandas as pd
import re

# --- STEP 1: Force Environment Repair (Most Critical Step) ---
print("⚙️ Initializing environment (Downloading Google Chrome)...")

# 1. Uninstall conflicting versions
os.system("apt-get remove -y google-chrome-stable chromium-browser chromium-chromedriver")

# 2. Install dependencies
os.system("apt-get update")
os.system("apt-get install -y wget unzip")

# 3. Download and install official Google Chrome (.deb)
if not os.path.exists("google-chrome-stable_current_amd64.deb"):
    os.system("wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb")

os.system("apt-get install -y ./google-chrome-stable_current_amd64.deb")
print("✅ Chrome installation complete!")

# 4. Install python libraries
try:
    import webdriver_manager
except ImportError:
    os.system("pip install webdriver-manager")

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# --- STEP 2: Scraper Logic ---
def run_scraper():
    print("🚀 Starting browser...")

    # Configure Chrome
    chrome_options = Options()
    chrome_options.add_argument('--headless') # Headless mode
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--window-size=1920,1080')
    # Spoof User Agent
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

    try:
        # Install driver using automatic manager
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        print("🔗 Connection successful! Starting capture of Carsome...")

    except Exception as e:
        print(f"❌ Browser launch failed: {e}")
        return

    all_cars = []
    base_url = "https://www.carsome.my/buy-car"
    pages = 3 # Scrape page count (Suggested 3 pages for demo to avoid timeout)

    for page in range(1, pages + 1):
        url = f"{base_url}?pageNo={page}"
        print(f"   📄 Processing page {page}/{pages}...")

        try:
            driver.get(url)
            # Wait for load
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, "mod-b-card")))

            # 🔥 Simulate human scrolling (Trigger lazy loading images)
            for i in range(1, 6):
                driver.execute_script(f"window.scrollTo(0, {i * 600});")
                time.sleep(0.5)

            cards = driver.find_elements(By.CLASS_NAME, "mod-b-card")
            print(f"      -> Found {len(cards)} cars")

            for card in cards:
                try:
                    # 1. Title
                    title = card.find_element(By.CLASS_NAME, "mod-b-card__title").text

                    # 2. Price
                    try: price = card.find_element(By.CLASS_NAME, "mod-card__price").text
                    except: price = "RM 0"

                    # 3. Detail text (Used to extract mileage and transmission)
                    details = card.find_elements(By.CLASS_NAME, "mod-b-card__car-other")
                    detail_text = " ".join([d.text for d in details])

                    mileage = "0 km"
                    match = re.search(r'[\d,]+\s*km', detail_text)
                    if match: mileage = match.group(0)

                    trans = "Automatic"
                    if "Manual" in detail_text: trans = "Manual"

                    # 4. Location
                    try:
                        loc = card.find_element(By.CLASS_NAME, "mod-b-card__car-location").text
                    except:
                        loc = "Malaysia"

                    # 5. Image (Prioritize finding data-src)
                    img = ""
                    try:
                        img_elem = card.find_element(By.CSS_SELECTOR, ".mod-b-card__header img")
                        img = img_elem.get_attribute("data-src") or img_elem.get_attribute("src")
                    except: pass

                    # 6. Link
                    link = ""
                    try:
                        link = card.find_element(By.TAG_NAME, "a").get_attribute("href")
                    except: pass

                    # Store data
                    all_cars.append({
                        'mod-b-card__title': title,
                        'mod-card__price 2': price,
                        'mod-b-card__car-other': mileage,
                        'mod-b-card__car-other 2': trans,
                        'mod-b-card__car-location': loc,
                        'mod-b-card__header href': img,
                        'last-swiper href': link,
                        'mod-b-card__mask-content': price
                    })

                except Exception as e:
                    continue # Skip broken cards

        except Exception as e:
            print(f"   ⚠️ Error loading page {page}: {e}")

    driver.quit()

    # --- STEP 3: Save Data ---
    if all_cars:
        df = pd.DataFrame(all_cars)
        # Fill in potentially missing columns
        for col in ['mod-b-card__title', 'mod-card__price 2', 'mod-b-card__car-other',
                    'mod-b-card__car-other 2', 'mod-b-card__car-location']:
            if col not in df.columns: df[col] = ""

        df.to_csv("carsome_test.csv", index=False)
        print("="*40)
        print(f"✅ Scraping complete! Total {len(df)} cars.")
        print("📂 Data saved to: carsome_test.csv")
        print("👉 Now go run your Dashboard!")
        print("="*40)
    else:
        print("❌ Scraping failed, possibly network blocking.")

# Start
run_scraper()

⚙️ 正在初始化环境 (下载 Google Chrome)...
✅ Chrome 安装完成！
🚀 正在启动浏览器...
🔗 连接成功！开始抓取 Carsome...
   📄 正在处理第 1/3 页...
      -> 找到 18 辆车
   📄 正在处理第 2/3 页...
      -> 找到 18 辆车
   📄 正在处理第 3/3 页...
      -> 找到 18 辆车
✅ 抓取完成！共 54 辆车。
📂 数据已保存至: carsome_test.csv
👉 现在去运行你的 Dashboard 吧！


In [ ]:
import pandas as pd
import numpy as np
import gradio as gr
import urllib.parse
import re
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib.ticker import FuncFormatter

# --- 1. Global Configuration ---
sns.set(style="whitegrid")
plt.switch_backend("Agg")
# PREMIUM_PALETTE retained for other charts
PREMIUM_PALETTE = ["#2c3e50", "#e67e22", "#e74c3c", "#3498db", "#9b59b6", "#2ecc71"]
sns.set_palette(PREMIUM_PALETTE)

# --- 2. Data Loading & Smart Cleaning ---
df_raw = pd.DataFrame()
try:
    df_raw = pd.read_csv("carsome_test.csv")
    df_raw = df_raw.astype(str)
    df_raw.fillna("", inplace=True)
    print(f"✅ CSV read successfully: Original rows {len(df_raw)}")
except:
    print("⚠️ Warning: Cannot read CSV file")

df_display = pd.DataFrame()
if not df_raw.empty:
    # Title
    title_col = next((c for c in df_raw.columns if 'title' in c.lower()), None)
    df_display['Title'] = df_raw[title_col] if title_col else 'Unknown'

    # Smart Price Extraction (Smart Regex) - Fix double price issue
    price_col = 'mod-card__price 2' if 'mod-card__price 2' in df_raw.columns else None
    if not price_col:
        price_col = next((c for c in ['mod-card__price', 'price', 'Price'] if c in df_raw.columns), None)

    print(f"ℹ️ Locked price column: {price_col}")

    def extract_first_price(row):
        val = str(row.get(price_col, ""))
        # Extract only the digit after the first RM, ignore promotional prices
        if "RM" in val:
            match = re.search(r'RM\s*([\d,]+)', val)
            if match:
                return match.group(1).replace(',', '')
        return "0"

    df_display['PriceNum'] = df_raw.apply(extract_first_price, axis=1)
    df_display['PriceNum'] = pd.to_numeric(df_display['PriceNum'], errors='coerce').fillna(0)

    # Save raw text
    df_display['PriceRaw'] = df_raw[price_col] if price_col else ""

    # Monthly Payment Calculation
    df_display['EstMonthly'] = df_display['PriceNum'].apply(lambda x: (x*0.9 + x*0.9*0.035*9)/(9*12) if x > 0 else 0)

    # Mileage
    other_col = next((c for c in df_raw.columns if 'other' in c.lower()), None)
    if other_col:
        df_display['Mileage'] = df_raw[other_col]
        df_display['MileageNum'] = pd.to_numeric(df_display['Mileage'].str.replace(r'[^\d]', '', regex=True), errors='coerce').fillna(0)
    else:
        df_display['Mileage'] = "N/A"
        df_display['MileageNum'] = 0

    # Location Cleaning
    loc_col = next((c for c in df_raw.columns if 'location' in c.lower()), 'mod-b-card__car-location')
    raw_loc = df_raw.get(loc_col, pd.Series(['Malaysia']*len(df_raw)))

    def get_state_bucket(loc_str):
        loc = str(loc_str).lower().strip()
        if re.match(r'^\d', loc): return "Other"
        for s in ['kuala lumpur', 'selangor', 'johor', 'penang', 'perak', 'melaka', 'negeri sembilan', 'pahang', 'kedah', 'sabah', 'sarawak', 'putrajaya']:
            if s in loc: return s.title()
        return "Other"
    df_display['State'] = raw_loc.apply(get_state_bucket)

    def get_detailed_city(loc_str):
        s = str(loc_str).strip()
        if re.match(r'^\d', s): return "Unknown Area"
        if ',' in s: return s.split(',')[0].strip().title()
        return s.title()
    df_display['City'] = raw_loc.apply(get_detailed_city)

    # Transmission
    def get_trans(row):
        row_str = str(row.values).lower()
        if 'manual' in row_str: return 'Manual'
        return 'Automatic'
    df_display['Transmission'] = df_raw.apply(get_trans, axis=1)

    # Link
    link_col = next((c for c in df_raw.columns if 'href' in c.lower()), None)
    df_display['Link'] = df_raw[link_col].apply(lambda x: str(x) if str(x).startswith('http') else "https://www.carsome.my"+str(x)) if link_col else "#"

    # Year & Brand
    def extract_meta(title):
        t = str(title).strip()
        year_match = re.search(r'^(\d{4})', t)
        year = int(year_match.group(1)) if year_match else 2020
        clean_t = re.sub(r'^\d{4}\s+', '', t)
        brand = clean_t.split()[0].title() if clean_t else "Other"
        if re.search(r'[\d\.]', brand): brand = "Other"
        return pd.Series([year, brand, clean_t])
    df_display[['Year', 'Brand', 'CleanTitle']] = df_display['Title'].apply(extract_meta)

    # SVG
    def generate_svg_url(brand_name):
        clean_name = str(brand_name).strip().upper()
        brand = clean_name.split()[0] if clean_name else "CAR"
        if re.search(r'[\d\.]', brand): brand = "CAR"
        colors = {'H': '#1abc9c', 'T': '#e74c3c', 'P': '#9b59b6', 'B': '#34495e', 'M': '#f1c40f'}
        bg_color = colors.get(brand[0], '#95a5a6')
        svg_html = f"""<svg width="400" height="250" xmlns="http://www.w3.org/2000/svg"><defs><linearGradient id="grad1" x1="0%" y1="0%" x2="100%" y2="100%"><stop offset="0%" style="stop-color:{bg_color};stop-opacity:1" /><stop offset="100%" style="stop-color:#ecf0f1;stop-opacity:1" /></linearGradient></defs><rect width="100%" height="100%" fill="url(#grad1)"/><text x="50%" y="50%" font-family="Segoe UI, sans-serif" font-weight="900" font-size="42" fill="rgba(255,255,255,0.4)" dominant-baseline="middle" text-anchor="middle">{brand}</text><text x="50%" y="50%" font-family="Segoe UI, sans-serif" font-weight="bold" font-size="38" fill="#2c3e50" dominant-baseline="middle" text-anchor="middle">{brand}</text></svg>"""
        return f"data:image/svg+xml;utf8,{urllib.parse.quote(svg_html)}"
    df_display['ImageURL'] = df_display['CleanTitle'].apply(generate_svg_url)

else:
    for c in ['Title', 'PriceRaw', 'PriceNum', 'EstMonthly', 'Mileage', 'MileageNum', 'State', 'City', 'Link', 'Year', 'Brand', 'ImageURL', 'Transmission']:
        df_display[c] = []

# Keep only valid price data
df_clean = df_display[df_display['PriceNum'] > 0].copy()

# --- 3. Coordinate Dictionary ---
malaysia_coords = {
    "Kuala Lumpur": {"lat": 3.1390, "lon": 101.6869}, "Selangor": {"lat": 3.3000, "lon": 101.5183}, "Johor": {"lat": 2.0000, "lon": 103.3000},
    "Penang": {"lat": 5.4141, "lon": 100.3288}, "Perak": {"lat": 4.6921, "lon": 101.0901}, "Kedah": {"lat": 6.1184, "lon": 100.6685},
    "Melaka": {"lat": 2.2500, "lon": 102.2501}, "Negeri Sembilan": {"lat": 2.7258, "lon": 102.2000}, "Pahang": {"lat": 3.8126, "lon": 102.8000},
    "Terengganu": {"lat": 5.0000, "lon": 103.0000}, "Kelantan": {"lat": 5.8000, "lon": 102.1000}, "Perlis": {"lat": 6.5500, "lon": 100.2500},
    "Sabah": {"lat": 5.9788, "lon": 116.5000}, "Sarawak": {"lat": 2.5000, "lon": 113.0000}, "Putrajaya": {"lat": 2.9264, "lon": 101.6964}, "Other": {"lat": 4.2105, "lon": 101.9758}
}

# --- 4. Chart Factory ---
def plot_chart(chart_type, data):
    if data.empty: return None

    fig, ax = plt.subplots(figsize=(8, 4.5))

    if chart_type == "dist":
        sns.histplot(data['PriceNum'], bins=25, kde=True, color="#1abc9c", edgecolor="white", alpha=0.7, ax=ax)
        mean_val = data['PriceNum'].mean()
        median_val = data['PriceNum'].median()
        ax.axvline(mean_val, color='#e74c3c', linestyle='--', linewidth=2, label=f'Mean: {mean_val/1000:.1f}k')
        ax.axvline(median_val, color='#2c3e50', linestyle=':', linewidth=2, label=f'Median: {median_val/1000:.1f}k')
        ax.xaxis.set_major_formatter(FuncFormatter(lambda x, pos: f'{int(x/1000)}k'))
        ax.set_title(f"Price Distribution Analysis (Avg: RM {mean_val:,.0f})", fontsize=11, fontweight='bold')
        ax.set_xlabel("Price (RM)")
        ax.legend()

    elif chart_type == "scatter":
        sns.scatterplot(x='MileageNum', y='PriceNum', data=data, hue='Year', palette='viridis', ax=ax)
        ax.set_title("Price vs Mileage")

    elif chart_type == "trend":
        avg = data.groupby('Year')['PriceNum'].mean().reset_index()
        sns.lineplot(x='Year', y='PriceNum', data=avg, marker='o', linewidth=2.5, color="#e67e22", ax=ax)
        ax.set_title("Depreciation Trend")

    elif chart_type == "brand_vol":
        cnt = data['Brand'].value_counts().head(8)
        sns.barplot(x=cnt.values, y=cnt.index, palette="mako", ax=ax)
        ax.set_title("Top 8 Brands Volume")

    elif chart_type == "brand_price":
        avg = data.groupby('Brand')['PriceNum'].mean().sort_values(ascending=False).head(8)
        sns.barplot(x=avg.values, y=avg.index, palette="rocket", ax=ax)
        ax.set_title("Most Expensive Brands")

    # 🔥🔥🔥 Fix Focus: State Pie Chart 🔥🔥🔥
    elif chart_type == "state_pie":
        d_loc = data[data['State'] != "Other"]
        if d_loc.empty: d_loc = data
        loc_counts = d_loc['State'].value_counts().head(7)

        # 1. Use a palette with more colors (tab10 has 10 colors) to ensure 7 states don't repeat
        n_slices = len(loc_counts)
        distinct_colors = sns.color_palette("tab10", n_slices)

        # 2. Draw pie chart and capture return objects (wedges, texts, autotexts)
        wedges, texts, autotexts = ax.pie(
            loc_counts.values,
            labels=loc_counts.index,
            autopct='%1.1f%%',
            startangle=140,
            colors=distinct_colors, # Use new palette
            wedgeprops={'edgecolor': 'white', 'linewidth': 1} # Add white border for better texture
        )

        # 3. Key: Set percentage text (autotexts) to white and bold for high contrast
        plt.setp(autotexts, size=10, weight="bold", color="white")

        ax.set_title("Regional Share (Top States Pie)")

    elif chart_type == "year_count":
        sns.countplot(x='Year', data=data, palette="viridis", ax=ax)
        ax.set_title("Inventory by Year")

    plt.tight_layout()
    return fig

# --- Map (Plotly) ---
def plot_map(data):
    state_counts = data['State'].value_counts().reset_index()
    state_counts.columns = ['State', 'Count']
    state_counts = state_counts[state_counts['Count'] > 0]
    state_counts = state_counts[state_counts['State'] != "Other"]

    def get_coords(state):
        coords = malaysia_coords.get(state, malaysia_coords["Other"])
        return pd.Series([coords['lat'], coords['lon']])

    state_counts[['Lat', 'Lon']] = state_counts['State'].apply(get_coords)
    state_counts['Label'] = state_counts['State'] + "\n(" + state_counts['Count'].astype(str) + ")"

    fig = px.scatter_geo(
        state_counts, lat="Lat", lon="Lon", size="Count", color="State", text="Label", size_max=40,
        title="Malaysia Vehicle Distribution (Map)", color_discrete_sequence=px.colors.qualitative.Bold, projection="mercator"
    )
    fig.update_traces(mode="markers+text", textposition="top center", textfont=dict(size=11, color="black"))
    fig.update_geos(visible=False, showcoastlines=True, coastlinecolor="#cccccc", showland=True, landcolor="#f9f9f9", fitbounds=False, lataxis_range=[0.5, 7.5], lonaxis_range=[99.5, 119.5])
    fig.update_layout(margin={"r":0,"t":30,"l":0,"b":0})
    return fig

# --- 5. Core Logic ---
def update_analytics(brand, year, state, trans, max_price):
    d = df_clean.copy()

    if brand != "All": d = d[d['Brand'] == brand]
    if year != "All": d = d[d['Year'] == int(year)]
    if state != "All": d = d[d['State'] == state]
    if trans != "All": d = d[d['Transmission'] == trans]

    if max_price <= 300000:
        d = d[d['PriceNum'] <= max_price]

    if d.empty: return ["No Data"] * 10

    metrics = f"RM {d['PriceNum'].mean():,.0f}"
    count = f"{len(d)} Units"

    return (metrics, count,
            plot_chart("dist", d), plot_chart("scatter", d),
            plot_chart("trend", d), plot_chart("brand_vol", d),
            plot_chart("brand_price", d), plot_map(d),
            plot_chart("state_pie", d), plot_chart("year_count", d))

def render_card(row):
    return f"""
    <div class="card">
        <div class="card-header"><img src="{row['ImageURL']}" alt="car"><div class="badge">{row['Year']}</div></div>
        <div class="card-body">
            <h4>{row['CleanTitle']}</h4>
            <div class="price-tag">{row['PriceRaw']}</div>
            <div class="monthly-tag">Est. RM {row['EstMonthly']:,.0f} / mo</div>
            <div class="specs"><span>📍 {row['City']}</span><span>🚗 {row['Mileage']}</span></div>
            <a href="{row['Link']}" target="_blank" class="btn">View Details</a>
        </div>
    </div>
    """

def simple_search(query, sort_by):
    d = df_clean.copy()
    if query: d = d[d['Title'].str.lower().str.contains(query.lower())]
    if sort_by == "Price: Low to High": d = d.sort_values("PriceNum", ascending=True)
    elif sort_by == "Price: High to Low": d = d.sort_values("PriceNum", ascending=False)
    elif sort_by == "Year: Newest": d = d.sort_values("Year", ascending=False)
    elif sort_by == "Mileage: Low to High": d = d.sort_values("MileageNum", ascending=True)
    if d.empty: return "<div style='padding:20px'>No vehicles found matching criteria.</div>"
    cards = "".join(d.apply(render_card, axis=1))
    return f'<div class="grid">{cards}</div>'

def loan_calculator(price, down_payment_pct, rate, years):
    principal = price * (1 - down_payment_pct/100)
    total_interest = principal * (rate/100) * years
    total_loan = principal + total_interest
    monthly = total_loan / (years * 12)
    return f"""<div style="background:#f8f9fa; padding:20px; border-radius:10px; border-left: 5px solid #2ecc71;"><h3>💰 Result</h3><p>Price: <b>RM {price:,.2f}</b></p><p>Loan: <b>RM {principal:,.2f}</b></p><p>Interest: <b>RM {total_interest:,.2f}</b></p><hr><h2 style="color:#27ae60;">RM {monthly:,.2f} <small>/ mo</small></h2></div>"""

# --- 8. UI Setup ---
def clean_menu(lst):
    valid = [x for x in lst if x and x != "Other" and not re.match(r'^\d', str(x))]
    return ["All"] + sorted(valid)

brand_opts = clean_menu(df_clean['Brand'].unique().tolist())
year_opts = ["All"] + sorted(df_clean['Year'].unique().tolist())
state_opts = clean_menu(df_clean['State'].unique().tolist())
trans_opts = clean_menu(df_clean['Transmission'].unique().tolist())

css = """
.grid { display: grid; grid-template-columns: repeat(auto-fill, minmax(260px, 1fr)); gap: 20px; padding: 10px; }
.card { background: white; border-radius: 12px; box-shadow: 0 4px 15px rgba(0,0,0,0.05); overflow: hidden; border: 1px solid #eee; transition: transform 0.3s ease; }
.card:hover { transform: translateY(-5px); box-shadow: 0 10px 25px rgba(0,0,0,0.1); }
.card-header { height: 140px; position: relative; overflow: hidden; }
.card-header img { width: 100%; height: 100%; object-fit: cover; }
.badge { position: absolute; top: 10px; right: 10px; background: rgba(0,0,0,0.7); color: white; padding: 2px 8px; border-radius: 4px; font-size: 12px; }
.card-body { padding: 15px; }
h4 { margin: 0 0 5px 0; font-size: 15px; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; color: #2c3e50; }
.price-tag { color: #e74c3c; font-weight: 800; font-size: 18px; }
.monthly-tag { font-size: 12px; color: #27ae60; font-weight: bold; margin-bottom: 10px; }
.specs { display: flex; justify-content: space-between; font-size: 12px; color: #7f8c8d; margin-bottom: 15px; }
.btn { display: block; width: 100%; padding: 10px 0; background: #34495e; color: white; text-align: center; border-radius: 6px; text-decoration: none; font-weight: 600; }
.btn:hover { background: #2c3e50; }
"""

with gr.Blocks(css=css, theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🏎️ Carsome Analytics ")

    with gr.Tabs():
        with gr.Tab("📊 Analytics"):
            with gr.Row():
                dd_brand = gr.Dropdown(brand_opts, value="All", label="Brand")
                dd_year = gr.Dropdown(year_opts, value="All", label="Year")
                dd_state = gr.Dropdown(state_opts, value="All", label="State (Filter)")
            with gr.Row():
                dd_trans = gr.Dropdown(trans_opts, value="All", label="Transmission")
                sld_price = gr.Slider(minimum=10000, maximum=300000, value=300000, step=10000, label="Max Budget (RM)")
            btn_refresh = gr.Button("Analyze", variant="primary")

            with gr.Row():
                kpi_price = gr.Textbox(label="Avg Price")
                kpi_count = gr.Textbox(label="Count")

            # Row 1
            with gr.Row():
                p1 = gr.Plot(label="Price Dist")
                p2 = gr.Plot(label="Price vs Mileage")
            # Row 2
            with gr.Row():
                p3 = gr.Plot(label="Depreciation Trend")
                p4 = gr.Plot(label="Brand Volume")
            # Row 3
            with gr.Row():
                p5 = gr.Plot(label="Most Expensive Brands")
                p6 = gr.Plot(label="Regional Distribution (Map)")
            # Row 4
            with gr.Row():
                p7 = gr.Plot(label="Regional Share (Pie Fix)") # Fixed Pie Chart
                p8 = gr.Plot(label="Inventory by Year (Count)")

            btn_refresh.click(update_analytics,
                              inputs=[dd_brand, dd_year, dd_state, dd_trans, sld_price],
                              outputs=[kpi_price, kpi_count, p1, p2, p3, p4, p5, p6, p7, p8])

        with gr.Tab("🔍 Search"):
            with gr.Row():
                txt_q = gr.Textbox(label="Model", placeholder="e.g. Honda Civic")
                dd_sort = gr.Dropdown(["Relevance", "Price: Low to High", "Price: High to Low", "Year: Newest", "Mileage: Low to High"], value="Relevance", label="Sort By")
                btn_find = gr.Button("Search", variant="primary")
            html_res = gr.HTML(value=simple_search("", "Relevance"))
            btn_find.click(simple_search, [txt_q, dd_sort], html_res)

        with gr.Tab("🧮 Calculator"):
            with gr.Row():
                in_price = gr.Number(label="Price", value=50000)
                in_down = gr.Slider(0, 50, value=10, label="Down Payment (%)")
                in_rate = gr.Slider(1, 10, value=3.5, label="Interest (%)")
                in_year = gr.Slider(1, 9, value=9, step=1, label="Years")
            btn_calc = gr.Button("Calculate")
            out_calc = gr.HTML()
            btn_calc.click(loan_calculator, [in_price, in_down, in_rate, in_year], out_calc)

app.launch(share=True)

✅ CSV read successfully: Original rows 900
ℹ️ Locked price column: mod-card__price 2


/tmp/ipython-input-521094721.py:308: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft()) as app:
/tmp/ipython-input-521094721.py:308: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css, theme=gr.themes.Soft()) as app:


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ff0bb363e50bba98dd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
